In [6]:
1+2

3

In [18]:
import ollama
import numpy as np
from numpy.linalg import norm

def get_embedding(text):
    response = ollama.embeddings(model='embeddinggemma:300m', prompt=text)
    return np.array(response['embedding'])

def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

# 1. Generate embeddings
king = get_embedding("king")
man = get_embedding("man")
woman = get_embedding("woman")
queen = get_embedding("queen")

# 2. Perform the arithmetic: King - Man + Woman
# We are essentially "subtracting" the concept of masculinity 
# and "adding" the concept of femininity to the concept of royalty.
masculinity_vector = man - woman
result_vector = king - masculinity_vector

# 3. Verify the result
similarity = cosine_similarity(result_vector, queen)

print(f"Similarity between (King - Man + Woman) and Queen: {similarity:.4f}")

Similarity between (King - Man + Woman) and Queen: 0.7030


In [20]:
import torch
import numpy as np
from transformers import AutoModel, AutoTokenizer
from typing import List, Tuple, Optional

# 1. SETUP: Load the model directly
# We use a small model (GPT-2) here for speed, but you can use 'google/gemma-2b'
# or 'meta-llama/Llama-2-7b' if you have the RAM.
MODEL_NAME = "qwen3:0.6b"

print(f"Loading {MODEL_NAME}...")
tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model: PreTrainedModel = AutoModel.from_pretrained(MODEL_NAME)

# 2. THE "SMART" PART: Access the full Embedding Matrix
# This matrix contains the vector for EVERY token in the vocabulary.
# Shape: [Vocab_Size, Hidden_Dim] (e.g., [50257, 768])
embedding_matrix: torch.Tensor = model.get_input_embeddings().weight.detach()


# Helper: Extract the raw vector for a single word
def get_embedding(word: str) -> Tuple[torch.Tensor, int]:
    # We allow the model to execute the lookup for us
    inputs = tokenizer(word, return_tensors="pt")

    # In many tokenizers, simple words might be split (e.g., "king" -> ["king"]).
    # We take the first meaningful token ID.
    token_id = inputs["input_ids"][0][0]

    # Look up the vector directly from the matrix
    vector = embedding_matrix[token_id]
    return vector, token_id


# 3. GET VECTORS
print("Fetching vectors...")
king_vec, king_id = get_embedding("king")
man_vec, man_id = get_embedding("man")
woman_vec, woman_id = get_embedding("woman")

# 4. DO THE MATH
# King - Man + Woman
# Note: We work with PyTorch Tensors now, not just lists
target_vector = king_vec - man_vec + woman_vec


# 5. FIND THE CLOSEST NEIGHBOR (The "Unembedding")
def find_closest_tokens(
    vector: torch.Tensor,
    matrix: torch.Tensor,
    k: int = 5,
    ignore_ids: Optional[List[int]] = None,
) -> List[Tuple[str, float]]:
    """
    Finds the top-k closest tokens in the entire matrix using Cosine Similarity.
    """
    # Normalize the target vector and the matrix for Cosine Similarity
    # Cosine Sim = (A . B) / (|A| * |B|)
    # If we normalize A and B first, it becomes just (A . B)

    vector_norm = vector / vector.norm()
    matrix_norm = matrix / matrix.norm(dim=1, keepdim=True)

    # Calculate scores for ALL words at once using Matrix Multiplication
    # Shape: [1, 768] @ [768, 50257] = [1, 50257] scores
    scores = torch.matmul(vector_norm, matrix_norm.T)

    # Sort the scores
    top_scores, top_indices = torch.topk(scores, k + len(ignore_ids))

    results = []
    for score, idx in zip(top_scores, top_indices):
        idx = idx.item()
        if idx in ignore_ids:
            continue

        word = tokenizer.decode([idx]).strip()
        results.append((word, score.item()))

        if len(results) >= k:
            break

    return results


# Exclude the input words themselves so we don't just find "King" again
ignore = [king_id.item(), man_id.item(), woman_id.item()]

print(f"\nMath: King - Man + Woman")
matches = find_closest_tokens(target_vector, embedding_matrix, k=5, ignore_ids=ignore)

print("\nClosest matches in the entire vocabulary:")
for i, (word, score) in enumerate(matches):
    print(f"{i+1}. {word} (Similarity: {score:.4f})")

/home/max/workspace/github.com/vegerot/localllm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading qwen3:0.6b...


HFValidationError: Repo id must use alphanumeric chars, '-', '_' or '.'. The name cannot start or end with '-' or '.' and the maximum length is 96: 'qwen3:0.6b'.